In [ ]:
# ==============================
# REQUIRED LIBRARIES
# ==============================
import os
import nibabel as nib
import numpy as np
import pandas as pd
import scipy.stats as stats
from dipy.io.image import load_nifti
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import TensorModel
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn import datasets

In [ ]:
# ==============================
# DEFINES DATA DIRECTORIES AND CONFIGURATIONS
# ==============================
DATA_DIR = "../data/raw"

OUTPUT_DIR = "../data/features"
os.makedirs(OUTPUT_DIR, exist_ok=True)

SAMPLE_SUBJECT = "sub-01" # For debugging

# Loads a standardized brain atlas (Harvard-Oxford Atlas),
# which divides the brain into predefined regions.
atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
masker = NiftiLabelsMasker(labels_img = atlas.maps, standardize=True)

In [ ]:
# ==============================
# FUNCTION: EXTRACTS T1W and T2W FEATURES BY REGION
# ==============================
def extract_t1_t2_features(subject_id):
    features = {"subject": subject_id}

    for scan_type in ["T1w", "T2w"]: # Loops over both T1-weighted and T2-weighted scans
        scan_path = os.path.join(DATA_DIR, subject_id, "anat", f"{subject_id}_{scan_type}.nii.gz")

        if os.path.exists(scan_path):
            img = nib.load(scan_path)

            # Extracts region-wise mean signal intensity using the Harvard-Oxford Atlas
            region_data = masker.fit_transform(img) 
            
            # Stores region-wise mean signal intensity
            for i, region_value in enumerate(region_data.flatten()):
                features[f"{scan_type}_region_{i}_mean"] = np.mean(region_value)

        else:
            print(f"Missing {scan_type} for {subject_id}")

    return features

In [ ]:
# ==============================
# SAMPLE FEATURE EXTRACTION - DEBUGGING
# ==============================
sample_t1_t2_features = extract_t1_t2_features(SAMPLE_SUBJECT)
print(sample_t1_t2_features)

In [ ]:
# ==============================
# FUNCTION: EXTRACTS DWI FEATURES BY REGION
# ==============================
def extract_dwi_features(subject_id):
    features = {"subject": subject_id}

    # All three file paths are needed for diffusion-weighted imaging (DWI) data
    dwi_path = os.path.join(DATA_DIR, subject_id, "dwi", f"{subject_id}_acq-lr_dwi.nii.gz")
    bval_path = os.path.join(DATA_DIR, subject_id, "dwi", f"{subject_id}_acq-lr_dwi.bval")
    bvec_path = os.path.join(DATA_DIR, subject_id, "dwi", f"{subject_id}_acq-lr_dwi.bvec")

    if os.path.exists(dwi_path):
        data, affine = load_nifti(dwi_path)
        bvals = np.loadtxt(bval_path) # Loads b-values (gradient strength) as bvals
        bvecs = np.loadtxt(bvec_path) # Loads b-vectors (gradient direction) as bvecs
        gtab = gradient_table(bvals, bvecs) # A gradient_table helps standardize DWI data across scanner machines

        # Fitting the model - needed in order to make fa, md, rd, and ad calculations.
        model = TensorModel(gtab)
        tensor_fit = model.fit(data)

        # Fractional Anisotropy (FA) - Measures how directional water diffusion is
        # Higher FA -> More structured white matter
        fa_regions = masker.fit_transform(nib.Nifti1Image(tensor_fit.fa, affine)) 
        # Mean Diffusivity (MD) - Measures overall diffusion in all directions
        # Higher MD -> More free water diffusion (vs restricted water diffusion)
        md_regions = masker.fit_transform(nib.Nifti1Image(tensor_fit.md, affine)) 
        # Radial Diffusivity (RD) - Measures diffusion perpendicular to white matter fibers
        # Lower RD -> More intact myelin
        rd_regions = masker.fit_transform(nib.Nifti1Image(tensor_fit.rd, affine)) 
        # Axial Diffusivity (AD) - Measures diffusion along the main fiber direction
        # Higher AD -> Stronger axonal integrity
        ad_regions = masker.fit_transform(nib.Nifti1Image(tensor_fit.ad, affine))

        # Stores mean diffusion measures per region
        for i in range(fa_regions.shape[1]): 
            features[f"dwi_region_{i}_fa_mean"] = np.mean(fa_regions[:, i])
            features[f"dwi_region_{i}_md_mean"] = np.mean(md_regions[:, i])
            features[f"dwi_region_{i}_rd_mean"] = np.mean(rd_regions[:, i])
            features[f"dwi_region_{i}_ad_mean"] = np.mean(ad_regions[:, i])
            
    else:
        print(f"Missing DWI files for {subject_id}")

    return features
        

In [ ]:
# ==============================
# SAMPLE FEATURE EXTRACTION - DEBUGGING
# ==============================
sample_dwi_features = extract_dwi_features(SAMPLE_SUBJECT)
print(sample_dwi_features)

In [ ]:
# ==============================
# FUNCTION: EXTRACTS FMRI FEATURES BY REGION
# ==============================
def extract_fmri_features(subject_id):
    features = {"subject": subject_id}

    fmri_path = os.path.join(DATA_DIR, subject_id, "func", f"{subject_id}_task-rest_run-01_bold.nii.gz")
    
    if os.path.exists(fmri_path):
        img = nib.load(fmri_path)

        # Fits the atlas to the image data
        time_series = masker.fit_transform(img)

        # Stores statistical metrics for each atlas region
        for i in range(time_series.shape[1]):
            features[f"fmri_region_{i}_mean_activation"] = np.mean(time_series[:, i])
            features[f"fmri_region_{i}_skew_activation"] = stats.skew(time_series[:, i])
            features[f"fmri_region_{i}_kurtosis_activation"] = stats.kurtosis(time_series[:, i])

        # Creates a connectivity matrix - a matrix where values represent the 
        # connectedness of two brain regions. 
        conn_measure = ConnectivityMeasure(kind="correlation")
        connectivity_matrix = conn_measure.fit_transform([time_series])[0]

        # Stores only the upper triangle of the connectivity matrix
        num_regions = connectivity_matrix.shape[0]
        for i in range(num_regions):
            for j in range(i+1, num_regions):
                features[f"conn_{i}_{j}"] = connectivity_matrix[i, j]

    else:
        print(f"Missing fMRI for {subject_id}")

    return features

In [ ]:
# ==============================
# SAMPLE FEATURE EXTRACTION - DEBUGGING
# ==============================
sample_fmri_features =  extract_fmri_features(SAMPLE_SUBJECT)
print(sample_fmri_features)

In [ ]:
# ==============================
# EXTRACTS ALL FEATURES FROM FIRST 5 SUBJECTS
# ==============================
subject_list = sorted([sub for sub in os.listdir(DATA_DIR) if sub.startswith("sub-")])[:5]

# Extracts features for each subject
all_features = []

for subject in subject_list:
    print(f"Extracting features for {subject}:")
    features = {}
    features.update(extract_t1_t2_features(subject))
    print(f"1/3 completed.")
    features.update(extract_dwi_features(subject))
    print(f"2/3 completed.")
    features.update(extract_fmri_features(subject))
    print(f"3/3 completed.")
    all_features.append(features)
    print(f"Subject {subject} data extraction completed.")

# Saves to CSV
df_features = pd.DataFrame(all_features)
output_file = os.path.join(OUTPUT_DIR, "first_5_subjects_features.csv")
df_features.to_csv(output_file, index=False)
print(f"Feature extraction complete. Data saved to: {output_file}")

In [ ]:
# ==============================
# EXTRACTS ALL FEATURES FROM ALL SUBJECTS
# ==============================
all_features = []

for subject in sorted(os.listdir(DATA_DIR)):
    if subject.startswith("sub-"):
        print(f"Extracting features for {subject}:")
        features = {}
        features.update(extract_t1_t2_features(subject))
        print(f"1/3 completed.")
        features.update(extract_dwi_features(subject))
        print(f"2/3 completed.")
        features.update(extract_fmri_features(subject))
        print(f"3/3 completed.")
        all_features.append(features)
        print(f"Subject {subject} data extraction completed.")

# Saves to CSV
df_features = pd.DataFrame(all_features)
output_file = os.path.join(OUTPUT_DIR, "extracted_features.csv")
df_features.to_csv(output_file, index=False)
print(f"Feature extraction complete. Data saved to: {output_file}")

In [ ]:
# ==============================
# EXTRACTS SCORE DATA & MERGES WITH BRAIN SCAN FEATURES
# ==============================
# File paths
SCOREDATA_FILE = os.path.join(DATA_DIR, "scoredata.csv")
FEATURES_FILE = os.path.join(OUTPUT_DIR, "extracted_features.csv")
OUTPUT_FILE = os.path.join(OUTPUT_DIR, "data.csv")

# Creates pandas dataframe for participant score data
participants_data_path = os.path.join(DATA_DIR, "participants.tsv")
df_scores = pd.read_csv(participants_data_path, sep="\t")

# Cleans spaces
df_scores["participant_id"] = df_scores["participant_id"].str.strip()

# Loads extracted_features.csv as a pandas dataframe
df_features = pd.read_csv(FEATURES_FILE)

# Renames 'subject' to 'participant_id' - needed for datasets to be joined
df_features.rename(columns = {"subject": "participant_id"}, inplace=True)

# Merges the two datasets on 'participant_id'
df_final = df_scores.merge(df_features, on="participant_id", how="inner")

# Saves to CSV
df_final.to_csv(OUTPUT_FILE, index = False)
print(f"Merged data saved to: {OUTPUT_FILE}")